In [2]:
import pandas as pd
import glob
import os
from gensim.models.keyedvectors import KeyedVectors
from gensim import matutils
from sklearn.ensemble import RandomForestClassifier as classifier
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [3]:
files = glob.glob(os.path.join("../../data/result", "*.csv"))
df_list = []

In [4]:
for file in files:
    tmp_df = pd.read_csv(file)
    tmp_df['filename'] = os.path.basename(file)
    df_list.append(tmp_df)

In [5]:
df = pd.concat(df_list, ignore_index=True)

In [6]:
# df.groupby('pred').count().sort_values(['obj'], ascending=False)[1:10]

In [7]:
# predの要素で, propertyが含まれる値を返す
df_prop = df.query('pred.str.contains("property")', engine='python')
top_10_df_prop = df_prop.groupby('pred').count().sort_values(['obj'], ascending=False).head(10)
top_10_prop_list = top_10_df_prop.index.to_list()
top_10_df = df[df['pred'].isin(top_10_prop_list)]

In [8]:
model = KeyedVectors.load('../../Models/japanese-word2vec-model-builder/word2vec.gensim.model')

In [9]:
def vectorize(model, word):
    try:
        output = model.wv[word]
        return output
    except:
        return "Null"

In [10]:
def preprocessing(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("：")) != 1:
        output = obj.split("：")[-1]
    else:
        output = obj.split("/")[-1]
    return output

In [12]:
# vectorization
cp_top_10_df = top_10_df.copy()
cp_top_10_df['key_vec'] = cp_top_10_df['key'].map(lambda x:vectorize(model,x))
#cp_top_10_df['label'] = cp_top_10_df['pred'].map(prop_to_label)
obj_list = cp_top_10_df['obj'].map(lambda x:preprocessing(x))
cp_top_10_df['obj_vec'] = obj_list.map(lambda x:vectorize(model,x))

In [13]:
cp_top_10_df

,key,pred,obj,filename,key_vec,obj_vec
12,沖縄県,http://ja.dbpedia.org/property/after,http://ja.dbpedia.org/resource/琉球列島米国軍政府,沖縄県.csv,"[0.03427477, 0.17860307, -0.041606415, 0.09745...","[-0.13429342, -0.09761905, 0.068803005, 0.0482..."
13,沖縄県,http://ja.dbpedia.org/property/after,-----,沖縄県.csv,"[0.03427477, 0.17860307, -0.041606415, 0.09745...",Null
14,沖縄県,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/琉球列島米国民政府,沖縄県.csv,"[0.03427477, 0.17860307, -0.041606415, 0.09745...","[-0.08374447, 0.07326782, -0.10170767, 0.10177..."
15,沖縄県,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/琉球政府,沖縄県.csv,"[0.03427477, 0.17860307, -0.041606415, 0.09745...","[-0.050090242, 0.18710567, -0.044265315, 0.181..."
16,沖縄県,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/琉球藩,沖縄県.csv,"[0.03427477, 0.17860307, -0.041606415, 0.09745...","[-0.20956942, -0.06843233, -0.041769836, -0.07..."
...,...,...,...,...,...,...
56783,高知県,http://ja.dbpedia.org/property/歌など,県の歌：高知県民の歌（1953年制定）,高知県.csv,"[-0.021919195, 0.10427152, 0.04427553, 0.12341...",Null
56784,高知県,http://ja.dbpedia.org/property/歌など,県の魚：カツオ,高知県.csv,"[-0.021919195, 0.10427152, 0.04427553, 0.12341...","[-0.1669918, 0.011190996, 0.08135691, 0.178282..."
56788,高知県,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/大分県,高知県.csv,"[-0.021919195, 0.10427152, 0.04427553, 0.12341...","[0.01619406, 0.14829713, 0.05118972, 0.1368117..."
56789,高知県,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/徳島県,高知県.csv,"[-0.021919195, 0.10427152, 0.04427553, 0.12341...","[0.013457059, 0.049274415, 0.034320407, 0.1147..."


In [14]:
top_10_df_vec = pd.merge(top_10_df,cp_top_10_df, left_index=True, right_on=top_10_df.index)
top_10_df_vec = top_10_df_vec.reset_index(drop=True)

In [15]:
top_10_df_vec = top_10_df_vec.drop(["key_0","filename_x","key_y","pred_y","obj_y","filename_y"], axis=1)

In [16]:
top_10_df_vec_exclusion_null = top_10_df_vec[top_10_df_vec['obj_vec'] != "Null"].reset_index(drop=True)

/Users/uehararyouta/Desktop/DBpedia_access_python/.venv/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [240]:
input_data = top_10_df_vec_exclusion_null['obj_vec'].values
label_data = top_10_df_vec_exclusion_null['label']

In [251]:
X_train, X_test, y_train, y_test = train_test_split(input_data, label_data, test_size=0.4)

In [260]:
top_10_df_vec_exclusion_null['label']

0      1000
1       300
2       300
3       300
4       800
       ... 
645     700
646     400
647     200
648     200
649     200
Name: label, Length: 650, dtype: int64

In [253]:
#clf = classifier(random_state=0).fit(X_train,y_train)